In [4]:
# we could use also tensor flow, there are multiple implementations of word2vec
from gensim.models import word2vec
import pandas as pd
import json
from pathlib import Path

In [5]:
#word to vec exploration

In [6]:
#first load all recipe data
recipes = []
ingredients = []
ids = []
path = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(path.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        recipes.append(r)
        ing = r.get('Ingredients')
        id = r.get('RecipeID')
        ids.append(id)
        if ing is not None:
            ingredients.append([i['Name'] for i in ing if i is not None and i['Name'] is not None])
       

/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-51-46-372714.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-108-181-2437430.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-13-123-101172.json
/Users/bridgitboulahanis/Dro

In [7]:
# Set values for NN parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 3    # 50% of the corpus                    
num_workers = 4       # Number of CPUs
context = 10          # Context window size; 
downsampling = 1e-3   # threshold for configuring which 
                    # higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(ingredients, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [8]:
ingredient = 'feta cheese'
df = pd.DataFrame([i for i in ingredients if ingredient in i])
w2v_ingredient = ingredients[1]
w2v_ingredient = 'feta'
print("testing", w2v_ingredient)
similar_ings = [similar_ing for similar_ing, _ in model.wv.most_similar(w2v_ingredient)]

invers_similar_ings = {}
for si in similar_ings:
    for isi, _ in model.wv.most_similar(si):
        invers_similar_ings.setdefault(isi, 0)
        invers_similar_ings[isi] += 1
    
print()
print("inverse", sorted(invers_similar_ings))

print("counts")
for i in sorted(list(invers_similar_ings.items()), key=lambda v: v[1])[-10:]:
    print(i)

print()
print("most similar", model.wv.most_similar(list(invers_similar_ings)))

print("negative")
model.wv.most_similar(negative=w2v_ingredient)

testing feta

inverse [' ', ' Freshly ground black pepper  ', ' Kosher salt  ', ' olive oil', ' �1', ' �2', 'Aluminum foil', 'Coarse salt and freshly ground pepper', 'Filling: ', 'Granny Smith apple', 'Kosher salt ', 'Sauce: ', 'Stock, Chicken', 'White Wine ', 'Worcestershire ', 'all-purpose flour  ', 'anchovies', 'basil  ', 'bay', 'beef stock ', 'brie cheese', 'butter  ', 'celery stick', 'cherry or grape tomatoes', 'chicken breasts ', 'chorizo sausage', 'courgettes', 'dry white wine ', 'each)', 'egg yolks ', 'feta', 'fresh garlic ', 'fresh lemon', 'garlic   ', 'grated parmesan cheese ', 'green peas', 'ground nutmeg ', 'ground pepper ', 'honey  ', 'low-fat cottage cheese', 'pasta ', 'plum tomatoes ', 'pork chops ', 'red peppers', 'reduced-fat', 'sea salt to taste', 'shallots ', 'shredded mozzarella cheese ', 'single cream', 'soy sauce  ', 'sugar  ', 'sundried tomatoes', 'tin chopped tomatoes', 'tofu ', 'turkey stock', 'vegetable bouillon', 'white wine  ', 'whole turkey', '�1']
counts
(

KeyError: "word 'f' not in vocabulary"